<h3> Import the necessary packages

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from yahoo_historical import Fetcher


<h3> Read the Apple stock data

In [2]:
ticker = 'QQQ'
start = [2019,1,1]

df = Fetcher(ticker, start).getHistorical()	
#df = df[['Date','Close']]
df.Date = pd.to_datetime(df.Date)

In [3]:
df.shape

(424, 7)

<h3> fundamentals

In [4]:
import pandas as pd
from searchtweets import load_credentials, gen_rule_payload, collect_results

In [5]:
premium_search_args = load_credentials("C:/Users/hadit/OneDrive - Prophix Software, Inc/search-tweets-python-master/.twitter_keys.yaml",
                                         account_type = "premium", env_overwrite=False)

C:\Users\hadit\AppData\Local\Continuum\anaconda3\lib\site-packages\searchtweets\credentials.py:34: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  search_creds = yaml.load(f)[yaml_key]
Grabbing bearer token from OAUTH


In [6]:
all_tweets=[]

In [9]:

rule = gen_rule_payload("stock market news (from:yahoofinance)",
                        from_date="2020-9-1", #UTC 2017-09-01 00:00
                        to_date="2020-9-8",#UTC 2017-10-30 00:00
                        results_per_call=100)
tweets = collect_results(rule,
                         max_results=100,
                         result_stream_args=premium_search_args) # change this if you need to


In [10]:
dates = [tweet.created_at_datetime for tweet in tweets]
dates = pd.to_datetime(dates, format="%Y-%m-%d")
print(dates.min())
print(dates.max())

2020-09-01 03:17:02
2020-09-07 22:19:03


In [11]:
len(tweets)

20

In [12]:
import datetime
dates.min().strftime('%Y-%m-%d')

'2020-09-01'

In [13]:
import pickle
fname = 'stock_market_' + str(dates.min().strftime('%Y-%m-%d')) + '-'+ str(dates.max().strftime('%Y-%m-%d')) +'.sav'
pickle.dump(tweets, open(fname,'wb'))

In [14]:
import os
import re

rootdir = "/mnt/externa/Torrents/completed"
regex = re.compile('(.*zip$)|(.*rar$)|(.*r01$)')

for root, dirs, files in os.walk(rootdir):
  for file in files:
    if regex.match(file):
       print(file)

In [15]:
len(tweets)

20

In [17]:
tweets

[{'created_at': 'Mon Sep 07 22:19:03 +0000 2020',
  'id': 1303095464603123712,
  'id_str': '1303095464603123712',
  'text': 'Why the explosive stock market rally is suddenly crumbling https://t.co/rKAh2EladZ by @BrianSozzi https://t.co/z1XSEUxxOU',
  'display_text_range': [0, 97],
  'source': '<a href="https://buffer.com" rel="nofollow">Buffer</a>',
  'truncated': False,
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'in_reply_to_screen_name': None,
  'user': {'id': 19546277,
   'id_str': '19546277',
   'name': 'Yahoo Finance',
   'screen_name': 'YahooFinance',
   'location': 'New York, NY',
   'url': 'http://bit.ly/2CvQXU1',
   'description': 'The planet’s biggest business news platform: https://finance.yahoo.com/\n\nGet the free Yahoo Finance app ⬇️',
   'translator_type': 'none',
   'protected': False,
   'verified': True,
   'followers_count': 977170,
   'friends_count': 360,
   'listed_count

In [16]:

all_tweets.append(tweets)
len(all_tweets)

NameError: name 'all_tweets' is not defined

In [ ]:
import pickle
pickle.dump(all_tweets, open('all_tweets2.sav','wb'))

In [ ]:
#all_tweets[0]

In [ ]:
text=[]
dates=[]
names=[]
for i in range(len(all_tweets)):
    text.append([tweet.all_text for tweet in all_tweets[i]])
    dates.append([tweet.created_at_datetime for tweet in all_tweets[i]])
    names.append([tweet.name for tweet in all_tweets[i]])

In [ ]:

text1 = [item for sublist in text for item in sublist]
news = pd.DataFrame(text1, columns={'text'})
news['Date'] = [item for sublist in dates for item in sublist]
news['name'] = [item for sublist in names for item in sublist]

In [ ]:
from datetime import timedelta

In [ ]:
news.Date = pd.to_datetime(news.Date)
news.Date = [a-timedelta(hours=4) for a in news.Date]
news = news.sort_values('Date').reset_index(drop=True)
news.to_csv('stock_market_news.csv',index=False)
news.head()

In [ ]:
text = [tweet.all_text for tweet in tweets]
news = pd.DataFrame(text, columns={'text'})
news['Date'] = [tweet.created_at_datetime for tweet in tweets]
news['name'] = [tweet.name for tweet in tweets]
news.to_csv('aapl_tweets2019b.csv',index=False)

In [ ]:
news = pd.read_csv('aapl_tweets2.csv')
news.head()

In [ ]:
news.dtypes

In [ ]:
import datetime as dt
news.Date = pd.to_datetime(news['Date']).dt.floor('d')
news.head()

In [ ]:
news.to_csv('news_aapl2.csv')

<h4> Apply sentiment

In [ ]:
from textblob import TextBlob

In [ ]:
text_blob = [TextBlob(i) for i in news.text]
text_polarity = [i.sentiment.polarity for i in text_blob]
text_subj = [i.sentiment.subjectivity for i in text_blob]

In [ ]:
news['sent_pol'] = text_polarity
news['sent_subj'] = text_subj
news.head()

In [ ]:
#news.to_csv('news_aapl.csv')

In [ ]:
news = pd.read_csv('news_aapl.csv')
news.head()

In [ ]:
news.shape

In [ ]:
news.Date = pd.to_datetime(news.Date)

In [ ]:
ticker = 'AAPL'
start = [2019,1,1]

df = Fetcher(ticker, start).getHistorical()	
#df = df[['Date','Close']]
df.Date = pd.to_datetime(df.Date)

In [ ]:
df['close_nextday'] = df.Close.shift(-1)
df['chg'] = (df.close_nextday-df.Close)>0
df.chg = df.chg.astype(int)
df.head()

In [ ]:
df2 = df.merge(news, on = 'Date')
df2.shape

In [ ]:
df2['sell2'] = df2.Close*1.01

In [ ]:
df2['sell_true2'] = df2.sell2 <= df2.high_nextday
df2

In [ ]:
df2.loc[df2.sell_true2 == False, 'sell2'] = df2.close_nextday
df2['gain_loss2'] = 100*(df2.sell2-df2.Close)/df2.Close
df2

In [ ]:
df2.to_csv('gain_loss_news.csv')

In [ ]:
df2.Close*1.01

In [ ]:
df2 = df1.merge(news[['Date','name']], on = 'Date')

In [ ]:
df2

In [ ]:
df[df.Date>=pd.to_datetime('2019-6-4')].head()